In [7]:
import numpy as np, cv2, sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, QFileDialog, QInputDialog, QSizePolicy, QSlider, QDialog, QFrame
from PyQt5.QtGui import QPixmap, QIcon, QImage, QPainter, QPen, QFont
from PyQt5.QtCore import QSize, Qt, QPoint, QEvent, QRect, QCoreApplication
from PIL import Image, ImageDraw, ImageFont


class ImageEditor(QWidget):
    def __init__(self):
        super().__init__()
        
        self.original_img = None # 원본 이미지
        self.img = None # 현재 화면에 표시되는 이미지
        self.temp_img = None # 누적 효과가 적용된 이미지

        self.active_slider = None  # 현재 활성화된 슬라이더
        self.confirm_button = None  # 확인 버튼
        self.cropping = False

        self.mosaic_mode = False
        
        self.drawing = False  # 그리기 상태 플래그
        self.drawing_mode = False  # 그리기 모드 활성화 여부

        self.last_point = None

        self.text = None  # 텍스트 삽입을 위한 속성
        self.text_mode = False  # 텍스트 삽입 모드 플래그

        self.morphing_mode = False
        self.morphing_backup = None  # 왜곡 작업 전 상태를 저장할 변수
        
        self.initUI()
        
    def initUI(self): # 초기 UI 설정
        self.layout = QVBoxLayout() # 전체 레이아웃을 수직으로 배치
        self.setLayout(self.layout)

        self.label = QLabel() # 이미지를 표시할 레이블 생성
        self.label.setAlignment(Qt.AlignCenter) # 레이블 내용을 중앙 정렬
        self.layout.addWidget(self.label) # 레이블을 메인 레이아웃에 추가

        self.hbox_layout1 = QHBoxLayout() # 첫 번째 가로 레이아웃
        self.hbox_layout2 = QHBoxLayout() # 두 번째 가로 레이아웃
        self.hbox_layout3 = QHBoxLayout() # 세 번째 가로 레이아웃
        
        # 버튼 리스트 정의
        self.buttons = ['가져오기', '저장하기', '왼쪽으로 회전', '오른쪽으로 회전', '상하 반전', '좌우 반전', '수평 맞추기', '자르기', '복원',
                        '밝기', '블러 처리', '선명도', '흑백', '그리기', '텍스트 삽입', '모자이크', '외곽 검출', '왜곡']

        # 버튼 생성 및 배치
        for idx, button in enumerate(self.buttons):
            btn = QPushButton(button) # 버튼 생성
            btn.setIcon(QIcon(f'tools/{button}.png')) # 각 버튼에 대응하는 아이콘 설정
            btn.setIconSize(QSize(50, 50)) # 크기
            btn.clicked.connect(self.click_buttons) # 함수 연결
            if idx < 6:
                self.hbox_layout1.addWidget(btn)
            elif idx >= 6 and idx < 12:
                self.hbox_layout2.addWidget(btn)
            else:
                self.hbox_layout3.addWidget(btn)
        # 가로 레이아웃을 메인 수직 레이아웃에 추가
        self.layout.addLayout(self.hbox_layout1)
        self.layout.addLayout(self.hbox_layout2)
        self.layout.addLayout(self.hbox_layout3)

        # 밝기 트랙바
        self.brightness_slider = QSlider(Qt.Horizontal)
        self.brightness_slider.setMinimum(0)
        self.brightness_slider.setMaximum(255)
        self.brightness_slider.setValue(127)
        self.brightness_slider.valueChanged.connect(self.apply_brightness)
        self.layout.addWidget(self.brightness_slider)
        self.brightness_slider.hide()

        # 블러 트랙바
        self.blur_slider = QSlider(Qt.Horizontal)
        self.blur_slider.setMinimum(0)
        self.blur_slider.setMaximum(100)
        self.blur_slider.setValue(0)
        self.blur_slider.valueChanged.connect(self.apply_blur)
        self.layout.addWidget(self.blur_slider)
        self.blur_slider.hide()

        # 선명도 트랙바
        self.sharpness_slider = QSlider(Qt.Horizontal)
        self.sharpness_slider.setMinimum(0)
        self.sharpness_slider.setMaximum(100)
        self.sharpness_slider.setValue(0)
        self.sharpness_slider.valueChanged.connect(self.apply_sharpness)
        self.layout.addWidget(self.sharpness_slider)
        self.sharpness_slider.hide()

        # 흑백 트랙바
        self.grayscale_slider = QSlider(Qt.Horizontal)
        self.grayscale_slider.setMinimum(0)
        self.grayscale_slider.setMaximum(100)
        self.grayscale_slider.setValue(0)
        self.grayscale_slider.valueChanged.connect(self.apply_grayscale)
        self.layout.addWidget(self.grayscale_slider)
        self.grayscale_slider.hide()

        # 팔레트 프레임 추가
        self.palette_frame = QFrame()
        self.palette_layout = QHBoxLayout()
        self.palette_frame.setLayout(self.palette_layout)
        self.layout.addWidget(self.palette_frame)
        self.palette_frame.hide()  # 초기에는 숨김

        # 팔레트 색상 버튼 추가
        self.colors = [
            ((255, 0, 0), "빨강"),  # 빨간색
            ((255, 165, 0), "주황"),  # 주황색
            ((255, 255, 0), "노랑"),  # 노란색
            ((0, 255, 0), "초록"),  # 초록색
            ((0, 255, 255), "청록"),  # 청록색
            ((0, 0, 255), "파랑"),  # 파란색
            ((128, 0, 128), "보라"),  # 보라색
            ((0, 0, 0), "검정"),  # 검정색
            ((255, 255, 255), "흰색")  # 흰색
        ]
        for color, name in self.colors:
            btn = QPushButton() # 각 색상에 대한 버튼 생성
            # 버튼의 배경색을 해당 색상으로 설정(OpenCV의 BGR 형식을 RGB로 변환)
            btn.setStyleSheet(f"background-color: rgb({color[2]}, {color[1]}, {color[0]});")
            btn.setFixedSize(30, 30)
            # 버튼 클릭 시 해당 색상을 그리기 색상으로 설정하는 함수 연결
            btn.clicked.connect(lambda _, c=color: self.set_draw_color(c))
            self.palette_layout.addWidget(btn)

        # 수평 맞추기 트랙바
        self.tilt_slider = QSlider(Qt.Horizontal)
        self.tilt_slider.setMinimum(-30)  # 왼쪽 최대 각도
        self.tilt_slider.setMaximum(30)   # 오른쪽 최대 각도
        self.tilt_slider.setValue(0)      # 초기값: 0도
        self.tilt_slider.valueChanged.connect(self.apply_tilt)
        self.layout.addWidget(self.tilt_slider)
        self.tilt_slider.hide()  # 초기에는 숨김

        # 확인 버튼(트랙바가 있는 기능들에 확인 버튼 추가)
        self.confirm_button = QPushButton("확인")
        self.confirm_button.clicked.connect(self.confirm_changes)
        self.layout.addWidget(self.confirm_button)
        self.confirm_button.hide()  # 초기에는 숨김

        self.setWindowTitle("포토샵 프로그램")
        self.setGeometry(100, 100, 800, 700)
        self.show()

    # 버튼 클릭 시 실행되는 함수
    def click_buttons(self):
        button_text = self.sender().text() # 클릭된 버튼의 텍스트 가져오기
        if button_text == '가져오기':
            fname = QFileDialog.getOpenFileName(self) # 파일 열기 대화상자를 표시하여 이미지 파일 선택
            if fname[0]: # 선택된 파일이 있을 경우
                self.open_image(fname)
        elif button_text == '저장하기':
            fname = QFileDialog.getSaveFileName(self, 'Save File', '', "Images (*.png *.xpm *.jpg *.bmp)") # 파일 저장 대화상자를 표시하여 저장할 파일 이름과 경로 선택
            if fname[0]:
                cv2.imwrite(fname[0], self.img)
        elif button_text == '왼쪽으로 회전':
            self.rotate_left()
        elif button_text == '오른쪽으로 회전':
            self.rotate_right()
        elif button_text == '상하 반전':
            self.flip_vertically()
        elif button_text == '좌우 반전':
            self.flip_horizontally()
        elif button_text == '수평 맞추기':
            self.toggle_slider(self.tilt_slider)
        elif button_text=='자르기':
            self.is_cropping = True
            self.crop_start = None
            self.drawing_mode = False  
            self.cropping = True 
            self.mosaic_mode = False
            self.morphing_mode = False
        elif button_text == '복원':
            self.restore_image()
        if button_text == '밝기':
            self.toggle_slider(self.brightness_slider)
        elif button_text == '블러 처리':
            self.toggle_slider(self.blur_slider)
        elif button_text == '선명도':
            self.toggle_slider(self.sharpness_slider)
        elif button_text == '흑백':
            self.toggle_slider(self.grayscale_slider)    
        elif button_text=='그리기':
            self.drawing_mode = not self.drawing_mode
            if self.drawing_mode:
                self.palette_frame.show()  # 팔레트 표시
            else:
                self.palette_frame.hide()  # 팔레트 숨기기
            self.cropping = False
            self.mosaic_mode = False
            self.morphing_mode = False
        elif button_text == '텍스트 삽입':
            self.drawing_mode = False
            self.palette_frame.setVisible(self.drawing_mode)  # 팔레트 표시/숨김
            self.cropping = False
            self.mosaic_mode = False
            self.text_mode = not self.text_mode  # 텍스트 모드 토글
            if self.text_mode:
                self.text, ok = QInputDialog.getText(self, '텍스트 삽입', '텍스트를 입력해주세요:')
                if not ok:
                    self.text = None  # 사용자가 입력 취소한 경우
            else:
                self.text = None  # 텍스트 모드 종료 시 초기화
            self.morphing_mode = False
        elif button_text == '모자이크':
            self.drawing_mode = False  
            self.cropping = False 
            self.mosaic_mode = True
            self.morphing_mode = False
        elif button_text == '외곽 검출':
            self.apply_canny()
        elif button_text == '왜곡':
            self.start_morphing_mode()
        #drawing과 crop과 mosaic와 stamp는 같은 마우스 이벤트 함수를 사용하므로 false, true로 구분

    # 이미지를 불러오는 함수
    def open_image(self, fname):
        self.img = cv2.imread(fname[0])
        if self.img is None: raise Exception("영상파일 읽기 에러")
        self.original_img = self.img.copy() # 이미지를 복원할 때 사용할 변수(이미지를 복사하여 저장)
        self.display_image(self.img) # 이미지를 화면에 불러오기

    # 이미지를 화면에 나타내는 함수
    def display_image(self,img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2.imread BGR 형태 -> RGB 형태 변환

        # OpenCV 이미지 데이터를 QImage 형식으로 변환(차례대로 이미지 데이터, 이미지 너비, 이미지 높이, 이미지의 한 줄 크기(바이트), RGB 포맷)
        qimg = QImage(img_rgb.data, img_rgb.shape[1], img_rgb.shape[0], img_rgb.strides[0], QImage.Format_RGB888)
        self.label.setPixmap(QPixmap.fromImage(qimg)) # QImage를 QPixmap으로 변환 후 QLabel에 설정하여 화면에 이미지 표시

    # 왼쪽으로 회전하는 함수
    def rotate_left(self):
        if self.img is not None:
            self.img = cv2.rotate(self.img, cv2.ROTATE_90_COUNTERCLOCKWISE) # rotate 함수를 사용하여 이미지를 반시계 방향으로 90도 회전
            self.display_image(self.img)

    # 오른쪽으로 회전하는 함수
    def rotate_right(self):
        if self.img is not None:
            self.img = cv2.rotate(self.img, cv2.ROTATE_90_CLOCKWISE) # rotate 함수를 사용하여 이미지를 시계 방향으로 90도 회전
            self.display_image(self.img)

    # 상하 반전 함수
    def flip_vertically(self):
        if self.img is not None:
            self.img = cv2.flip(self.img, 0) # 두 번째 인자로 0을 전달하면 상하 반전
            self.display_image(self.img)

    # 좌우 반전 함수
    def flip_horizontally(self):
        if self.img is not None:
            self.img = cv2.flip(self.img, 1) # 두 번째 인자로 1을 전달하면 좌우 반전
            self.display_image(self.img)

    # 이미지를 복원하는 함수
    def restore_image(self):
        if self.original_img is not None:
            self.img = self.original_img.copy()  # 원본 이미지를 현재 상태로 복원
            self.display_image(self.img)

            # 트랙바 값을 초기값으로 리셋
            self.brightness_slider.setValue(127)  # 밝기 초기값
            self.blur_slider.setValue(0)          # 블러 초기값
            self.sharpness_slider.setValue(0)     # 선명도 초기값
            self.grayscale_slider.setValue(0)     # 흑백 초기값
            self.tilt_slider.setValue(0)          # 수평 맞추기 각도 초기값

            
    # 슬라이더 값 변경 이벤트 처리
    def slider_changed(self):
        value = self.slider.value()
        if self.slider_action == '밝기':
            self.apply_brightness(value)
        elif self.slider_action == '블러 처리':
            self.apply_blur(value)
        elif self.slider_action == '선명도':
            self.apply_sharpness(value)
        elif self.slider_action == '흑백':
            self.apply_grayscale(value)
        elif self.slider_action == '외곽 검출':
            self.apply_canny(value)
        elif self.slider_action == '수평 맞추기':
            self.apply_tilt(value)

            
    # 트랙바 표시/숨김
    def toggle_slider(self, slider):
        sliders = [self.brightness_slider, self.blur_slider, self.sharpness_slider, self.grayscale_slider, self.tilt_slider]
        for s in sliders:
            if s == slider:
                s.setVisible(not s.isVisible())
                if s.isVisible():  # 슬라이더가 보이면 확인 버튼도 표시
                    self.confirm_button.show()
                    self.active_slider = slider  # 현재 활성 슬라이더 설정
            else:
                s.hide()
        if not slider.isVisible(): # 슬라이더가 숨겨지면 확인 버튼도 숨김
            self.confirm_button.hide()

    # 확인 버튼
    def confirm_changes(self):
        if self.active_slider:
            value = self.active_slider.value()

            # 슬라이더 값에 따라 이미지를 확정 적용
            if self.active_slider == self.brightness_slider:
                self.apply_brightness(value)
            elif self.active_slider == self.blur_slider:
                self.apply_blur(value)
            elif self.active_slider == self.sharpness_slider:
                self.apply_sharpness(value)
            elif self.active_slider == self.grayscale_slider:
                self.apply_grayscale(value)

            # 슬라이더와 확인 버튼 숨김
            self.active_slider.hide()
            self.confirm_button.hide()

            self.img = self.temp_img.copy()
            # 슬라이더와 확인 버튼 숨김
            if self.active_slider:
                self.active_slider.hide()
            self.confirm_button.hide()

    # 밝기 조절 함수
    def apply_brightness(self, value):
        if self.img is not None:
            adjustment = (value - 127) / 127  # 슬라이더 값이 127일 때 원본, 0일 때 어둡게, 255일 때 밝게

            # convertScaleAbs 함수로 밝기 조절   alpha는 밝기 조정 비율, beta는 추가 밝기 값
            self.temp_img = cv2.convertScaleAbs(self.img, alpha=1 + adjustment, beta=0)
            self.display_image(self.temp_img)

    # 블러 처리 함수
    def apply_blur(self, value):
        if self.img is not None and value > 0:
            kernel_size = 2 * value + 1 # 커널 크기는 홀수여야 하므로 +1
            # GaussianBlur 함수로 블러 효과 적용
            self.temp_img = cv2.GaussianBlur(self.img, (kernel_size, kernel_size), 0) # (kernel_size, kernel_size): 가우시안 블러의 커널 크기
            self.display_image(self.temp_img)

    # 선명도 조절 함수
    def apply_sharpness(self, value):
        if self.img is not None and value > 0:
            # 샤프닝 커널 생성
            # 중심 픽셀을 강조하여 주변 픽셀 대비를 증가시키는 역할
            kernel = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]], dtype=np.float32)
        
            # filter2D 함수로 샤프닝 필터 적용
            self.temp_img = cv2.filter2D(self.img, -1, kernel)
        
            # 슬라이더 값에 따라 샤프닝 강도 조절
            alpha = 1.0 + (value / 50.0)  # 슬라이더 값이 클수록 선명도가 높아짐
            self.temp_img = cv2.addWeighted(self.temp_img, alpha, self.img, 1 - alpha, 0) # addWeighted 함수로 두 이미지를 가중치(alpha) 비율로 합성
        
            # 화면에 출력
            self.display_image(self.temp_img)
        elif value == 0: # 슬라이더 값이 0이면 원본 이미지 출력
            self.temp_img = self.img.copy()
            self.display_image(self.temp_img)

    # 흑백 조절 함수
    def apply_grayscale(self, value):
        if self.img is not None:
            intensity = value / 100 # 흑백 효과의 강도 계산    value가 0이면 원본 이미지, 100이면 완전 흑백
            gray_img = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY) # 이미지를 그레이스케일로 변환
            gray_img_colored = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR) # 그레이스케일 이미지를 다시 BGR 포맷으로 변환(addWeighted에서 원본 이미지와 혼합할 수 있도록 하기 위해)
            self.temp_img = cv2.addWeighted(self.img, 1 - intensity, gray_img_colored, intensity, 0) # intensity에 따라 원본 이미지와 흑백 이미지를 가중치로 합성
            self.display_image(self.temp_img)

    # 수평 맞추기 함수(기울임)
    def apply_tilt(self, value):
        # 수평 맞추기 초기값
        self.tilt_angle = 0  # 현재 기울기 각도
        if self.img is not None:
            self.tilt_angle = value  # 슬라이더 값으로 기울기 각도 설정
        
            # 이미지 크기 계산
            height, width = self.img.shape[:2] # 이미지의 높이와 너비
            radian = np.radians(abs(self.tilt_angle)) # 각도를 라디안으로 변환
        
            # 기울기 각도에 따라 확대 비율 계산
            scale = 1 + abs(self.tilt_angle) / 45  # 각도 45도 기준 최대 2배 확대
        
            # 확대된 이미지 크기 계산
            new_w = int(width * scale)
            new_h = int(height * scale)

            # 이미지를 확대하여 왜곡 시 깎이는 영역 방지
            resized_img = cv2.resize(self.img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

            # 확대된 이미지의 중심 계산
            center = (new_w // 2, new_h // 2)

            # 회전 행렬 생성     회전 중심: 확대된 이미지의 중심
            matrix = cv2.getRotationMatrix2D(center, self.tilt_angle, 1.0)

            # 확대된 이미지 회전
            rotated_img = cv2.warpAffine(resized_img, matrix, (new_w, new_h))

            # 화면에 출력할 이미지 크롭 (원래 크기와 동일하게 유지)
            x_start = (new_w - width) // 2
            y_start = (new_h - height) // 2
            self.temp_img = rotated_img[y_start:y_start + height, x_start:x_start + width]
            
            self.display_image(self.temp_img)

    # 캐니 검출 함수(외곽)
    def apply_canny(self):
        if self.img is not None:
            # 캐니 검출 임계값 설정
            low_thresh = 100  # 하한값
            high_thresh = 200  # 상한값
        
            blur_img = cv2.GaussianBlur(self.img, (5, 5), 0) # 가우시안 블러 적용 (노이즈 제거)
            edges = cv2.Canny(blur_img, low_thresh, high_thresh) # 캐니 엣지 검출 적용
            self.temp_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) # 검출된 엣지를 RGB 이미지로 변환
        
            # 화면에 출력
            self.display_image(self.temp_img)

    # 팔레트 활성화 / 비활성화
    def toggle_draw_mode(self):
        if not self.palette_visible:
            self.show_palette()
        else:
            self.hide_palette()

    def set_draw_color(self, color):
        self.draw_color = color
        
    # 팔레트 보여주기
    def show_palette(self):
        self.palette_visible = True
        color = QColorDialog.getColor()
        if color.isValid():
            self.pen_color = (color.red(), color.green(), color.blue())

    # 팔레트 숨기기
    def hide_palette(self):
        self.palette_visible = False

    # 모자이크 크기 조절 메서드
    def update_mosaic_size(self):
        self.mosaic_size = self.mosaic_slider.value()  # 현재 트랙바 값을 모자이크 크기로 저장
        
    # 마우스를 눌렀을 때의 동작
    def mousePressEvent(self, event):
        if self.label.pixmap() is not None:
            # QLabel의 크기와 실제 이미지의 크기를 가져옴
            label_size = self.label.size()
            pixmap_size = self.label.pixmap().size()

            # 이벤트 위치를 이미지 좌표로 변환
            x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
            y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

            if 0 <= x < pixmap_size.width() and 0 <= y < pixmap_size.height():
                if event.button() == Qt.LeftButton:
                    if self.drawing_mode:  # 그리기 모드
                        self.drawing = True
                        # OpenCV가 QPoint 객체를 직접 처리할 수 없기 때문에 OpenCV의 함수들은 좌표를 (x, y) 형식의 튜플이나 리스트로 전달받아야 함.
                        self.last_point = (x, y)
                    elif self.cropping: #자르기 모드
                        self.crop_start = QPoint()
                        self.crop_end = QPoint()
                        self.crop_start = QPoint(x,y)
                    elif self.mosaic_mode:  # 모자이크 모드
                        self.mosaic_start = QPoint(x, y)
                    elif self.text_mode and self.text:  # 텍스트 삽입 모드
                        if self.text is not None:
                            self.img = cv2.putText(self.img, self.text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2) #putText 함수 활용
                            self.display_image(self.img)
                    elif self.morphing: # 왜곡 모드
                        self.pt1 = (x, y)
                elif event.button() == Qt.RightButton and event.type() == QEvent.MouseButtonDblClick:
                    if self.morphing_backup is not None:
                        self.img = self.morphing_backup.copy()  # 왜곡 전 상태로 복원
                        self.display_image(self.img)

    # 마우스를 움직였을 때의 동작
    def mouseMoveEvent(self, event):
        if self.label.pixmap() is not None:
            # QLabel의 크기와 실제 이미지의 크기를 가져옴
            label_size = self.label.size()
            pixmap_size = self.label.pixmap().size()

            # 이벤트 위치를 이미지 좌표로 변환
            x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
            y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

            if 0 <= x < pixmap_size.width() and 0 <= y < pixmap_size.height():
                if self.drawing: #그리기 모드
                    cv2.line(self.img, self.last_point, (x, y), self.draw_color, 2)
                    self.last_point = (x, y)
                    self.display_image(self.img)
                if self.cropping: #자르기 모드
                    self.crop_end = QPoint(x, y)
                    self.temp_img = self.img.copy()
                    cv2.rectangle(self.temp_img, (self.crop_start.x(), self.crop_start.y()), (x, y), (0, 0, 255), 1)
                    self.display_image(self.temp_img)
                if self.mosaic_mode:  # 모자이크 모드
                    self.mosaic_end = QPoint(x, y)
                    self.temp_img = self.img.copy()
                    cv2.rectangle(self.temp_img, (self.mosaic_start.x(), self.mosaic_start.y()), (x, y), (0, 255, 0), 2)
                    self.display_image(self.temp_img)

    # 마우스를 뗐을 때의 동작
    def mouseReleaseEvent(self, event):
        if self.label.pixmap() is not None:
            # QLabel의 크기와 실제 이미지의 크기를 가져옴
            label_size = self.label.size()
            pixmap_size = self.label.pixmap().size()

            # 이벤트 위치를 이미지 좌표로 변환
            x = event.pos().x() - (label_size.width() - pixmap_size.width()) // 2
            y = event.pos().y() - (label_size.height() - pixmap_size.height()) // 2

            if self.drawing and (event.button() == Qt.LeftButton): # 그리기 모드 종료 처리
                self.drawing = False
            if self.cropping: # 자르기 모드 종료 처리

                # 시작 및 종료 좌표 계산
                x_start, y_start = min(self.crop_start.x(), self.crop_end.x()), min(self.crop_start.y(), self.crop_end.y()) # 드래그 방향에 관계없이 사각형의 왼쪽 위 꼭짓점을 시작점으로 설정
                x_end, y_end = max(self.crop_start.x(), self.crop_end.x()), max(self.crop_start.y(), self.crop_end.y()) # 드래그 방향에 관계없이 사각형의 오른쪽 아래 꼭짓점을 종료점으로 설정

                # 좌표가 이미지 크기 내에 있는지 확인
                if 0 <= x_start < self.img.shape[1] and 0 <= y_start < self.img.shape[0] and 0 < x_end <= self.img.shape[1] and 0 < y_end <= self.img.shape[0]:
                    cropped_img = self.img[y_start:y_end, x_start:x_end]  # ROI 영역 추출
                    if cropped_img.size > 0: # 유효한 크기의 ROI만 처리
                        self.img = cv2.resize(cropped_img, (self.original_img.shape[1], self.original_img.shape[0]), interpolation=cv2.INTER_LINEAR) # 자른 이미지를 원본 이미지 크기로 리사이즈
                        self.display_image(self.img)
                self.cropping = False #자르기 모드 종료!
            if self.mosaic_mode: # 모자이크 모드 종료 처리

                # 시작 및 종료 좌표 계산
                x_start, y_start = min(self.mosaic_start.x(), self.mosaic_end.x()), min(self.mosaic_start.y(), self.mosaic_end.y()) # 드래그 방향에 관계없이 사각형의 왼쪽 위 꼭짓점을 시작점으로 설정
                x_end, y_end = max(self.mosaic_start.x(), self.mosaic_end.x()), max(self.mosaic_start.y(), self.mosaic_end.y()) # 드래그 방향에 관계없이 사각형의 오른쪽 아래 꼭짓점을 종료점으로 설정
                w, h = x_end - x_start, y_end - y_start # 선택 영역의 너비와 높이 계산

                # 좌표가 이미지 크기 내에 있는지 확인
                if 0 <= x_start < self.img.shape[1] and 0 <= y_start < self.img.shape[0] and 0 < x_end <= self.img.shape[1] and 0 < y_end <= self.img.shape[0]:
                    # ROI 영역 추출 및 모자이크 처리
                    roi = self.img[y_start:y_end, x_start:x_end]
                    roi = cv2.resize(roi, (w//15, h//15)) # 크기를 축소하여 픽셀화
                    roi = cv2.resize(roi, (w,h), interpolation=cv2.INTER_AREA) # 원래 크기로 복원
                    self.img[y_start:y_end, x_start:x_end] = roi # 원본 이미지에 적용
                    self.display_image(self.img)
                self.mosaic_mode = False # 모자이크 모드 종료
            if self.morphing_mode: # 왜곡 모드 종료 처리
                if self.pt1 and (event.button() == Qt.LeftButton):
                    self.pt2 = (x, y) # 드래그 종료점 설정
                    self.morphing_backup = self.img.copy() # 왜곡 전 상태 저장
                    self.morphing() # 왜곡 실행

    # 왜곡 모드를 활성화하는 함수
    def start_morphing_mode(self):
        self.morphing_mode = not self.morphing_mode # 왜곡 모드 상태 전환
    
        if self.morphing_mode: # 왜곡 모드 활성화
            self.pt1 = None
            self.pt2 = None
            self.label.mousePressEvent = self.mousePressEvent
            self.label.mouseReleaseEvent = self.mouseReleaseEvent
        else: # 왜곡 모드 비활성화
            self.pt1 = None
            self.pt2 = None
            self.label.mousePressEvent = None
            self.label.mouseReleaseEvent = None

    # 왜곡 효과를 보여주는 함수(드래그 거리만큼 영상 왜곡)
    def morphing(self):
        if self.pt1 is None or self.pt2 is None: return # 시작점 또는 종료점이 없으면 실행하지 않음

        h, w = self.img.shape[:2]
        dst = np.zeros((h, w, 3), self.img.dtype) # 반환할 이미지를 초기화 (검은색으로 채움)
        ys = np.arange(0, h, 1) # y 좌표 인덱스(1 간격)
        xs = np.arange(0, w, 0.1) # x 좌표 인덱스(0.1 간격)

        # 시작점의 x 좌표와 x 좌표를 0.1 간격으로 늘린 값 계산
        x1, x10 = self.pt1[0], self.pt1[0] * 10

        # 변경 비율 계산
        ratios = xs / x1 # 시작점 기준으로 변경 비율 계산
        ratios[x10:] = (w - xs[x10:]) / (w - x1) # x 좌표가 시작점 이후일 때 다른 비율 적용

        dxs = xs + ratios * (self.pt2[0] - self.pt1[0]) # 변경된 x 좌표 생성
        
        # 좌표를 정수값으로 변경, xs와 dxs를 이미지 크기 내로 제한
        xs = np.clip(xs.astype(int), 0, w - 1)
        dxs = np.clip(dxs.astype(int), 0, w - 1)

        # 원본과 변경된 좌표 행렬 생성
        ym, xm = np.meshgrid(ys, xs) # 원본 좌표 행렬
        _, dxm = np.meshgrid(ys, dxs) # 변경 좌표 행렬

        # 변경된 좌표에 원본 픽셀 값을 매핑
        dst[ym, dxm] = self.img[ym, xm]
        
        self.img = dst # 왜곡된 이미지를 업데이트
        self.display_image(self.img)

    # 키 누르는 이벤트(esc키 누르면 프로그램이 종료됨.)
    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Escape:  # ESC 키를 눌렀는지 확인
            self.close()  # 창 종료
            
if __name__ == "__main__":
    app = QCoreApplication.instance()
    if app is None: # 두 번째에 실행했을 때 커널이 죽는 문제를 방지하기 위해 일부 코드 수정
        app = QApplication(sys.argv)
    ex = ImageEditor()
    app.exec_()